Некоторые задания в этой тетрадки были созданы на основе соотвествующей тетрадки курса NLP от Elena Voita.

# Генерация текста: н-граммы

В этой тетрадке мы научимся делать простую модель генерации текста на основе н-грамм и встречаемости в корпусе.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [2]:
random.seed(1)
np.random.seed(1)

Будем пробовать генерировать шутки. Для обучения будем использовать [датасет с постами reddit](https://kaggle.com/datasets/thedevastator/one-million-reddit-jokes).

In [ ]:
path = # YOUR_PATH
data = pd.read_csv(path)

In [ ]:
data.head()

Так как наша задача генерации требует только текста, оставим только соответствующий столбец.

In [ ]:
columns = ['selftext']
data = data[columns]

## Обработка данных
###1. Чистка датасета
Для начала нужно определиться, есть в данных пропуски, и избавиться от них, если есть.

__Подсказка:__ Часто пропуски они обозначаются как nan, но иногда можно заметить иные способы.

In [ ]:
# <YOUR CODE HERE> #

Надо тексты привести к нижнему регистру. Пунктуацию можно оставить, так как она влияет на смысл предложения и на встречаемость.
Кроме этого можно избавиться от совсем коротких шуток, так как скорее всего это просто ответы на фразы. При делении текста на слова, используйте word_tokenize.

In [ ]:
from string import punctuation
import re

In [ ]:
def clean_text(text: str) -> list:
     '''
     Делит текст на слова и пунктуацию и приводит все к нижнему регистру
     :param text: строка
     :returns: список слов и знаков препинания
     '''
     # <YOUR CODE HERE> #
     return

In [ ]:
data['words'] = data['selftext'].apply(clean_text)
data['lens'] = data['words'].apply(len)
data = data[data.lens > 3]

In [ ]:
words = data['words'].tolist()

In [ ]:
words[:2]

[['my',
  'corona',
  'is',
  'covered',
  'with',
  'foreskin',
  'so',
  'it',
  'is',
  'not',
  'exposed',
  'to',
  'viruses',
  '.'],
 ["it's", 'called', 'google', 'sheets', '.']]

## N-grams
Для начала попробуем создать самую простую модель, основанную на встречаемости н-граммы в корпусе.

In [ ]:
from collections import defaultdict, Counter

In [ ]:
# добавляем токены начала и конца
BOS, EOS = '[bos]', '[eos]'

class NGramLanguageModel:
    def __init__(self, lines, n):
        assert n >= 1
        self.n = n
        counts = self.ngram_counts(lines, self.n)

        # перевести количества в вероятности
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        # <YOUR CODE HERE> #

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: строка запроса
        :returns: словарь с возможными продолжениями заданного префикса
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ BOS ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    @staticmethod
    def ngram_counts(lines: list, n: int) -> dict:
        '''
        Создаёт словарь, где каждому префиксу (n-1 слово) присваивается словарь,
        в котором ключи - слова, а значения - количество н-грамм в текстах
        :param lines: список списков
        :param n: количество слов в н-грамме
        :returns: словарь, в котором для каждого в префикса известно количество
        н-грамм с каждым словом
        '''
        dictionary = defaultdict(Counter)
        # dictionary[(word1, word2)][word3] = count((word1, word2, word3))
        # <YOUR CODE HERE> #
        return dictionary

# Проверим работу функции ngram_counts
dummy_lines = sorted(words, key=len)[:100]
dummy_counts = NGramLanguageModel.ngram_counts(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[(BOS, BOS)]) == 66
assert dummy_counts[BOS, 'a']['melon'] == 1

# Проверим работу модели
dummy_lm = NGramLanguageModel(dummy_lines, n=3)
p_initial = dummy_lm.get_possible_next_tokens('')
assert p_initial.most_common(1)[0][0] == 'a'

1. Попробуем составить предложение используя жадный метод.

In [ ]:
def get_next_word(lm: NGramLanguageModel, prefix: str) -> str:
    '''
    :param lm: language model
    :param prefix: строка префикса
    :returns: следующее, наиболее вероятное, слово для данного префикса
    '''
    # <YOUR CODE HERE> #
    return # next word

In [ ]:
lm = NGramLanguageModel(words, n=3)
prefix = 'get'
repeat = 20
for _ in range(repeat):
    word = get_next_word(lm, prefix)
    prefix += ' ' + word
    if prefix.endswith(EOS):
        break

print(prefix)

In [ ]:
prefix = ''
word = get_next_word(lm, prefix)
while word != EOS:
    prefix += f' {word}'
    word = get_next_word(lm, prefix)

print(prefix + f'{word} ')

2. Выбор наиболее вероятного слова не показал хороших результатов. Тем более, он будет строить очень много похожих предложений, а нам хочется разнообразия. Давайте попробуем семплировать методом top-k: выбираем k наиболее встречаемых вариантов и из них выбираем один случайным образом.

In [ ]:
def get_next_word(lm: NGramLanguageModel, prefix: str, k:int) -> str:
    '''
    :param lm: language model
    :param prefix: строка префикса
    :param k: количество слов в top-k
    :returns: следующее, наиболее вероятное, слово для данного префикса
    '''
    # <YOUR CODE HERE> #
    return # next word

In [ ]:
lm = NGramLanguageModel(words, n=3)
prefix = 'get'
repeat = 20
for i in range(repeat):
    word = get_next_word(lm, prefix, 5)
    prefix += ' ' + word
    if prefix.endswith(EOS):
        break

print(prefix)

In [ ]:
prefix = ''
word = get_next_word(lm, '', 5)
while word != EOS:
    prefix += f'{word} '
    word = get_next_word(lm, prefix, 5)
print(prefix + f'{word} ')

3. Для сравнения можно сделать beam search. Напоминаем, что он на каждом шаге выбирает k наилучших вариантов - те, с которыми наибольшая вероятность всего предложения. Кроме этого надо не забыть, что мы переводим вероятности в логарифмы: таким образом считается сумма логарифмов вероятностей для каждой н-граммы, из которого состоит предложение.
Чтобы не пересчитывать каждый раз корпус, давайте будем считать вероятность последней н-граммы.

Попробуйте написать свой beam search, для n-gramm, где n=3 и для k=2.

\begin{align}
        \frac{1}{L^\alpha}\sum_{t'=1}^LlogP(y_{t'}|y_{t'-n}, ..., y_{t'-1})
    \end{align}



In [ ]:
lm = NGramLanguageModel(words, n=3)
prefix = 'he' # YOUR IDEA
prefixes = [prefix, prefix]
probs = [0, 0]
k = 2

# <YOUR CODE HERE> #

## Оценивание модели
Раз мы научились делать простую языковую модель, надо понять, насколько она хорошо описывает язык. Для оценивания этого используем перплексию.

$PPL(W) = e^{ln(P(W)^{-\frac{1}{N}})} =e^{{-\frac{1}{N}}ln(P(W))}$



1. Для начала нужно разделить данные на тренировочные и тестовые

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data['words'], train_size=0.8)

2. Давайте напишем функцию, которая будет считать перплексию для каждого отдельного предложения на основе n-грамм. Для каждой нграммы считается её вероятность. Нам её считать не надо, так как она уже записана в атрибуте _self.probs_ в нашем классе модели.

In [ ]:
def perplexity(model: NGramLanguageModel, text: list, n=1) -> float:
    """
    :param model: language model
    :param text: список н-грамм предложения
    :param n: количество слов в н-грамме
    :returns: значение перплексии одного предложения
    """
    # <YOUR CODE HERE> #
    return

3. Теперь можем посчитать среднюю перплексию по всему датасету.

In [ ]:
n = 1
lm = NGramLanguageModel(X_train, n=n)
avg_ppl = 0
for sentence in X_test:
    test_sent = list(ngrams(sentence, n=n))
    value = perplexity(lm, test_sent, n=n)
    avg_ppl += value

avg_ppl /=  len(X_test)
assert np.isclose(avg_ppl, 1857.90, atol=1e-1)
print(avg_ppl)

Попробуем теперь сравнить перплексию для моделей униграм, биграм и триграм. Почему результаты получились такие?

In [ ]:
# <YOUR CODE HERE> #

## Использование готовых инструментов
К счастью, нам всё писать необязательно. Необходимые функции и модули уже были написаны умными программистами. Давайте посмотрим на модуль nltk и на то, что он нам предлагает.

In [ ]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, X_train)
model = MLE(n)
model.fit(train_data, padded_vocab)

In [ ]:
test_data, _ = padded_everygram_pipeline(n, X_test)

avg_ppl = 0
length = 0
for i, test in enumerate(test_data):
    # <YOUR CODE HERE> #
    pass

Давайте повторим то же самое но с моделями на биграммах и триграммах. Сравните результаты. Почему они такие?

In [ ]:
# <YOUR CODE HERE> #

И давайте напоследок попробуем сгенерировать последовательность с помощью обученной модели:

In [ ]:
model.generate(20, text_seed=['he'])

--------------

# Решение

## Обработка данных
###1. Чистка датасета

Для начала надо избавиться от пустых строк, или же нанов. Часто они обозначаются как nan, но иногда можно заметить иные способы.

In [30]:
path = '/content/drive/MyDrive/NLP_manual/Datasets/reddit_jokes.csv'
data = pd.read_csv(path)

In [31]:
data.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,ftbp1i,2qh72,jokes,False,1585785543,https://old.reddit.com/r/Jokes/comments/ftbp1i...,self.jokes,NaN,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,2
1,post,ftboup,2qh72,jokes,False,1585785522,https://old.reddit.com/r/Jokes/comments/ftboup...,self.jokes,NaN,It's called Google Sheets.,Did you know Google now has a platform for rec...,9
2,post,ftbopj,2qh72,jokes,False,1585785508,https://old.reddit.com/r/Jokes/comments/ftbopj...,self.jokes,NaN,The vacuum doesn't snore after sex.\r\n\r\n&am...,What is the difference between my wife and my ...,15
3,post,ftbnxh,2qh72,jokes,False,1585785428,https://old.reddit.com/r/Jokes/comments/ftbnxh...,self.jokes,NaN,[removed],My last joke for now.,9
4,post,ftbjpg,2qh72,jokes,False,1585785009,https://old.reddit.com/r/Jokes/comments/ftbjpg...,self.jokes,NaN,[removed],The Nintendo 64 turns 18 this week...,134


In [33]:
data = data[['selftext']]

In [34]:
data

,selftext
0,My corona is covered with foreskin so it is no...
1,It's called Google Sheets.
2,The vacuum doesn't snore after sex.\r\n\r\n&am...
3,[removed]
4,[removed]
...,...
999993,*zyan malik or whatever leaves 1d. \r\n*Kayne...
999994,[deleted]
999995,I'll be Bach
999996,So a moth goes into a podiatrists office.\r\n\...


In [35]:
data['selftext'].value_counts()[:10]

[removed]                          232919
[deleted]                          188442
\[removed\]                           272
To get to the other side.             125
Dr. Dre                               111
A stick.                               83
None.                                  81
A stick                                76
He worked it out with a pencil.        74
Then it hit me.                        72
Name: selftext, dtype: int64

Можно заметить, что наиболее частым классом являются _removed_ или _deleted_.

In [36]:
print('Размер данных до чистки', data.shape)
data = data[~data.isin(['[removed]', '[deleted]', '\[removed\]', 'removed', 'deleted'])]
data = data.dropna()
print('Размер данных после чистки', data.shape)

Размер данных до чистки (999998, 1)
Размер данных после чистки (573887, 1)


Надо тексты привести к нижнему регистру и убрать пунктуацию.
Кроме этого можно избавиться от совсем коротких шуток, так как скорее всего это просто ответы на фразы. При делении текста на слова, используйте word_tokenize.

In [37]:
from string import punctuation
import re
from nltk.tokenize import word_tokenize

In [39]:
def clean_text(text):
     text = text.lower()
     new_text = []
     for word in word_tokenize(text):
        new_text.append(word)
     return new_text

In [42]:
data['words'] = data['selftext'].apply(clean_text)
data['lens'] = data['words'].apply(len)
data = data[data.lens > 3]

In [43]:
words = data['words'].tolist()

In [44]:
words[:2]

[['my',
  'corona',
  'is',
  'covered',
  'with',
  'foreskin',
  'so',
  'it',
  'is',
  'not',
  'exposed',
  'to',
  'viruses',
  '.'],
 ['it', "'s", 'called', 'google', 'sheets', '.']]

## N-grams
Для начала попробуем создать самую простую модель, основанную на встречаемости н-граммы в корпусе.

In [45]:
from collections import defaultdict, Counter

In [47]:
# добавляем токены начала и конца
BOS, EOS = '[bos]', '[eos]'

class NGramLanguageModel:
    def __init__(self, lines, n):
        assert n >= 1
        self.n = n
        counts = self.ngram_counts(lines, self.n)

        # перевести количества в вероятности
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        for key, value in counts.items():
            sum_of_prefix = sum(value.values())
            for word, cnts in value.items():
                self.probs[key][word] = cnts / sum_of_prefix

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: строка запроса
        :returns: словарь с возможными продолжениями заданного префикса
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ BOS ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    @staticmethod
    def ngram_counts(lines, n):
        dictionary = defaultdict(Counter)
        for line in lines:
            new_line = [BOS] * (n-1) + line + [EOS]
            for i in range(n-1, len(new_line)):
                prefix = tuple(new_line[i-n+1:i])
                word = new_line[i]
                dictionary[prefix][word] += 1
        return dictionary

# Проверим работу функции ngram_counts
dummy_lines = sorted(words, key=len)[:100]
dummy_counts = NGramLanguageModel.ngram_counts(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[(BOS, BOS)]) == 59
assert dummy_counts[BOS, 'a']['melon'] == 1

# Проверим работу модели
dummy_lm = NGramLanguageModel(dummy_lines, n=3)
p_initial = dummy_lm.get_possible_next_tokens('')
assert p_initial.most_common(1)[0][0] == 'a'

1. Жадный метод.

In [ ]:
def get_next_word(lm, prefix):
    return lm.get_possible_next_tokens(prefix).most_common(1)[0][0]

In [ ]:
lm = NGramLanguageModel(words, n=3)
prefix = 'get'
repeat = 20
for _ in range(repeat):
    word = get_next_word(lm, prefix)
    prefix += ' ' + word
    if prefix.endswith(EOS):
        break

print(prefix)

get off the roof . [eos]


2. Top-k.

In [ ]:
def get_next_word(lm, prefix, k):
    next_words = lm.get_possible_next_tokens(prefix).most_common(k)
    index = random.randint(0, min(k, len(next_words))-1)
    return next_words[index][0]

In [ ]:
prefix = 'get'
repeat = 20
for _ in range(repeat):
    word = get_next_word(lm, prefix, k=5)
    prefix += ' ' + word
    if prefix.endswith(EOS):
        break

print(prefix)

get a divorce . the man replies : - ) ) [eos]


In [ ]:
prefix = ''
word = get_next_word(lm, prefix, k=5)
while word != EOS:
    prefix += f' {word}'
    word = get_next_word(lm, prefix, k=5)

print(prefix + f'{word} ')

 he was in my pocket and the bartender , who is a little bit of time . he says to the doctor .[eos] 


3. Beam search

In [49]:
lm = NGramLanguageModel(words, n=3)
prefix = 'he' # YOUR IDEA
prefixes = [prefix, prefix]
probs = [0, 0]
k = 2

step = 1
max_step = 20
while (not prefixes[0].endswith(EOS)) and (not prefixes[1].endswith(EOS)) and (step != max_step):
    print('step', step)
    print(prefixes, probs, sep='\n')
    step += 1
    for prefix in prefixes:
        possible_words1 = lm.get_possible_next_tokens(prefixes[0]).most_common(k)
    probs1 = []
    for word in possible_words1:
        probs1.append((word[0], probs[0]+np.log(word[1])))
    possible_words2 = lm.get_possible_next_tokens(prefixes[1]).most_common(k)
    probs2 = []
    for word in possible_words2:
        probs2.append((word[0], probs[0]+np.log(word[1])))
    choice = []
    probs1 = sorted(probs1, key=lambda x: x[1], reverse=True)
    probs2 = sorted(probs2, key=lambda x: x[1], reverse=True)
    probs_new = []
    while len(choice) != k and len(probs1) != 0 and len(probs2) != 0:
        if probs1[0][1] > probs2[0][1]:
            choice.append(prefixes[0] + f' {probs1[0][0]}')
            probs_new.append(probs1[0][1])
            probs1 = probs1[1:]
            possible_words1 = probs1[1:]
        else:
            choice.append(prefixes[1] + f' {probs2[0][0]}')
            probs_new.append(probs2[0][1])
            probs2 = probs2[1:]
            possible_words2 = probs2[1:]
    prefixes = choice
    probs = probs_new

step 1
['he', 'he']
[0, 0]
step 2
['he was', 'he was']
[-2.043168321757178, -2.043168321757178]
step 3
['he was a', 'he was a']
[-4.454801801110815, -4.454801801110815]
step 4
['he was a little', 'he was a little']
[-7.675854929925643, -7.675854929925643]
step 5
['he was a little bit', 'he was a little bit']
[-10.395961661061712, -10.395961661061712]
step 6
['he was a little bit of', 'he was a little bit of']
[-11.900688619406782, -11.900688619406782]
step 7
['he was a little bit of a', 'he was a little bit of a']
[-13.038863029384936, -13.038863029384936]
step 8
['he was a little bit of a sudden', 'he was a little bit of a sudden']
[-15.411522464771627, -15.411522464771627]
step 9
['he was a little bit of a sudden ,', 'he was a little bit of a sudden ,']
[-16.548124457982478, -16.548124457982478]
step 10
['he was a little bit of a sudden , the', 'he was a little bit of a sudden , a']
[-17.978663520900128, -17.978663520900128]
step 11
['he was a little bit of a sudden , the man', 'he w

## Оценивание модели
Раз мы научились делать простую языковую модель, надо понять, насколько она хорошо описывает язык. Для оценивания этого используем перплексию.

$PPL(W) = e^{ln(P(W)^{-\frac{1}{N}})} =e^{{-\frac{1}{N}}ln(P(W))}$

1. Для начала нужно разделить данные на тренировочные и тестовые

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data['words'], train_size=0.8)

2. Давайте напишем функцию, которая будет считать перплексию для каждого отдельного предложения на основе n-грамм. Для каждой нграммы считается её вероятность. Нам её считать не надо, так как она уже записана в атрибуте _self.probs_ в нашем классе модели.

In [ ]:
def perplexity(model, text, n=1):
    result = 0
    length = 0
    for ngram in text:
        if n == 1:
            prefix = ()
        else:
            prefix = ngram[:n-1]
        prob = model.probs[prefix][ngram[-1]]
        if prob != 0:
            result += np.log2(prob)
        length += 1
    enthropy = - (result / length)
    return 2**enthropy

3. Теперь можем посчитать среднюю перплексию по всему датасету.

In [ ]:
from nltk.util import ngrams

lm = NGramLanguageModel(X_train, n=1)
all_ppls_my = []
avg_ppl_my = 0
for sentence in X_test:
    test_sent = list(ngrams(sentence, n=1))
    value = perplexity(lm, test_sent, 1)
    all_ppls_my.append((' '.join(sentence), value))
    avg_ppl_my += value
print(avg_ppl_my / len(X_test))

1857.8977933685876


In [ ]:
s = sorted(all_ppls_my, key=lambda x: x[1])
print(s[0], s[-1])

('chs chsa rel smoth', 1.0) ('tak tak tak tak tak tak', 18871163.000000067)


Попробуем теперь сравнить перплексию для моделей униграм, биграм и триграм. Почему результаты получились такие?

In [ ]:
lm = NGramLanguageModel(X_train, n=2)
all_ppls = []
avg_ppl = 0
for sentence in X_test:
    test_sent = list(ngrams(sentence, n=2))
    value = perplexity(lm, test_sent, 2)
    all_ppls.append((' '.join(sentence), value))
    avg_ppl += value
print(avg_ppl / len(X_test))

89.18045429620587


In [ ]:
s = sorted(all_ppls, key=lambda x: x[1])
print(s[0], s[-1])

('caterpie metapod butterfree weedle kakuna beedrill paras parasect venonat venomoth scyther pinsir', 1.0) ("`` crack '' cocaine", 13670.813011882978)


In [ ]:
lm = NGramLanguageModel(X_train, n=3)
all_ppls = []
avg_ppl = 0
for sentence in X_test:
    test_sent = list(ngrams(sentence, n=3))
    value = perplexity(lm, test_sent, 3)
    all_ppls.append((' '.join(sentence), value))
    avg_ppl += value
print(avg_ppl / len(X_test))

11.931262401614887


In [ ]:
s = sorted(all_ppls, key=lambda x: x[1])
print(s[0], s[-1])

('ayy , el mayo !', 1.0) ('so , so .....', 2349.4520992492407)


## Использование готовых инструментов
К счастью, нам всё писать необязательно. Необходимые функции и модули уже были написаны умными программистами. Давайте посмотрим на модуль nltk и на то, что он нам предлагает.

In [ ]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, X_train)
model = MLE(n)
model.fit(train_data, padded_vocab)

In [ ]:
test_data, _ = padded_everygram_pipeline(n, X_test)
X_test_list = X_test.tolist()
all_ppls = []
avg_ppl = 0
length = 0
for i, test in enumerate(test_data):
    value = model.perplexity(test)
    all_ppls.append((' '.join(X_test_list[i]), value))
    if value != np.inf:
        avg_ppl += value
print(avg_ppl / i)

1737.34229072188


Давайте повторим то же самое но с моделями на биграммах и триграммах. Сравните результаты. Почему они такие?

In [ ]:
n = 2
train_data, padded_vocab = padded_everygram_pipeline(n, X_train)
model = MLE(n)
model.fit(train_data, padded_vocab)

test_data, _ = padded_everygram_pipeline(n, X_test)
all_ppls = []
avg_ppl = 0
length = 0
for i, test in enumerate(test_data):
    value = model.perplexity(test)
    all_ppls.append((' '.join(X_test_list[i]), value))
    if value != np.inf:
        avg_ppl += value
print(avg_ppl / i)

93.30178313379233


In [ ]:
s = sorted(all_ppls, key=lambda x: x[1])
print(s[0], s[-1])

("i do n't .", 31.782361206203593) ("'doctor , i 'm starting to think that i may be a hypochrondriac . ' the doctor let him know that this was okay .", inf)


In [ ]:
n = 3
train_data, padded_vocab = padded_everygram_pipeline(n, X_train)
model = MLE(n)
model.fit(train_data, padded_vocab)

test_data, _ = padded_everygram_pipeline(n, X_test)
all_ppls = []
avg_ppl = 0
length = 0
for i, test in enumerate(test_data):
    value = model.perplexity(test)
    all_ppls.append((' '.join(X_test_list[i]), value))
    if value != np.inf:
        avg_ppl += value
print(avg_ppl / i)

12.886113628097343


In [ ]:
s = sorted(all_ppls, key=lambda x: x[1])
print(s[0], s[-1])

("i do n't .", 14.247032242849127) ("'doctor , i 'm starting to think that i may be a hypochrondriac . ' the doctor let him know that this was okay .", inf)


И давайте напоследок попробуем сгенерировать последовательность с помощью обученной модели:

In [ ]:
model.generate(20, text_seed=['he'])

['asks',
 'his',
 'granfather',
 'for',
 'advice',
 '.',
 'sincerely',
 ',',
 '&',
 'amp',
 ';',
 'it',
 'was',
 'a',
 'free',
 'bed',
 ',',
 "''",
 'said',
 'paddy']